# Earth Model Comparison

This notebook demonstrates how to compare different 1D Earth models using the `seisray` package, focusing on velocity structures and their effects on seismic wave propagation.

## Learning Objectives
- Compare velocity structures of different Earth models
- Analyze differences in travel times between models
- Visualize velocity profiles and discontinuities
- Understand the impact of model choice on seismic analysis

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the parent directory to the path to import seisray
sys.path.append(os.path.dirname(os.getcwd()))

from seisray import TravelTimeCalculator, EarthModelManager, RayPathTracer
from obspy.taup import TauPyModel

print("Successfully imported seisray package!")

## 1. Available Earth Models

Let's start by exploring the available Earth models and their basic properties.

In [ ]:
# Create Earth model manager
manager = EarthModelManager()

# List available models
models = manager.list_available_models()
print(f"Available Earth models: {models}")

# Get detailed information about each model
for model in models:
    info = manager.get_model_info(model)
    print(f"\n{model.upper()} Model:")
    print(f"  Description: {info.get('description', 'N/A')}")
    print(f"  Depth range: {info['depth_range']} km")
    print(f"  Available phases: {len(info['phases'])} phases")
    print(f"  Example phases: {info['phases'][:8]}...")  # Show first 8 phases

## 2. Velocity Structure Comparison

Let's extract and compare the velocity structures of different models.

In [ ]:
# Function to extract velocity profile from TauPy model
def get_velocity_profile(model_name):
    model = TauPyModel(model=model_name)

    # Get the velocity model
    velocity_model = model.model

    depths = []
    vp_velocities = []
    vs_velocities = []

    for layer in velocity_model.s_velocity_layers:
        depths.append(layer.top_depth)
        vp_velocities.append(layer.top_p_velocity)
        vs_velocities.append(layer.top_s_velocity)

        depths.append(layer.bot_depth)
        vp_velocities.append(layer.bot_p_velocity)
        vs_velocities.append(layer.bot_s_velocity)

    return np.array(depths), np.array(vp_velocities), np.array(vs_velocities)

# Extract velocity profiles for all models
model_data = {}
for model in models:
    try:
        depths, vp, vs = get_velocity_profile(model)
        model_data[model] = {'depths': depths, 'vp': vp, 'vs': vs}
        print(f"Successfully extracted velocity profile for {model.upper()}")
    except Exception as e:
        print(f"Could not extract velocity profile for {model}: {e}")

In [ ]:
# Plot velocity profiles
fig, axes = plt.subplots(1, 2, figsize=(15, 8))
colors = ['blue', 'red', 'green']

# P-wave velocities
ax = axes[0]
for (model, data), color in zip(model_data.items(), colors):
    ax.plot(data['vp'], data['depths'], color=color, linewidth=2, label=model.upper())

ax.set_xlabel('P-wave Velocity (km/s)')
ax.set_ylabel('Depth (km)')
ax.set_title('P-wave Velocity Profiles')
ax.invert_yaxis()
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_ylim(0, 1000)  # Focus on upper 1000 km

# S-wave velocities
ax = axes[1]
for (model, data), color in zip(model_data.items(), colors):
    ax.plot(data['vs'], data['depths'], color=color, linewidth=2, label=model.upper())

ax.set_xlabel('S-wave Velocity (km/s)')
ax.set_ylabel('Depth (km)')
ax.set_title('S-wave Velocity Profiles')
ax.invert_yaxis()
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_ylim(0, 1000)  # Focus on upper 1000 km

plt.tight_layout()
plt.show()

## 3. Travel Time Differences Between Models

Now let's compare how different models predict travel times for P and S waves.

In [ ]:
# Set up parameters for comparison
source_depth = 10  # km
distances = np.linspace(10, 90, 17)  # degrees

# Calculate travel times for each model
travel_time_data = {}

for model in models:
    calc = TravelTimeCalculator(model)

    p_times = []
    s_times = []

    for distance in distances:
        arrivals = calc.calculate_travel_times(source_depth, distance)

        # Find P and S arrivals
        p_time = None
        s_time = None

        for arrival in arrivals:
            if arrival.name == 'P' and p_time is None:
                p_time = arrival.time
            elif arrival.name == 'S' and s_time is None:
                s_time = arrival.time

        p_times.append(p_time)
        s_times.append(s_time)

    travel_time_data[model] = {'p_times': np.array(p_times), 's_times': np.array(s_times)}
    print(f"Calculated travel times for {model.upper()}")

In [ ]:
# Plot travel time comparisons
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
colors = ['blue', 'red', 'green']

# P-wave travel times
ax = axes[0, 0]
for (model, data), color in zip(travel_time_data.items(), colors):
    ax.plot(distances, data['p_times']/60, color=color, linewidth=2,
           marker='o', markersize=4, label=model.upper())

ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('P-wave Travel Time (minutes)')
ax.set_title('P-wave Travel Time Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

# S-wave travel times
ax = axes[0, 1]
for (model, data), color in zip(travel_time_data.items(), colors):
    ax.plot(distances, data['s_times']/60, color=color, linewidth=2,
           marker='o', markersize=4, label=model.upper())

ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('S-wave Travel Time (minutes)')
ax.set_title('S-wave Travel Time Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

# P-wave time differences (relative to IASP91)
ax = axes[1, 0]
iasp91_p = travel_time_data['iasp91']['p_times']
for (model, data), color in zip(travel_time_data.items(), colors):
    if model != 'iasp91':
        diff = data['p_times'] - iasp91_p
        ax.plot(distances, diff, color=color, linewidth=2,
               marker='o', markersize=4, label=f'{model.upper()} - IASP91')

ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('P-wave Time Difference (s)')
ax.set_title('P-wave Time Differences (relative to IASP91)')
ax.legend()
ax.grid(True, alpha=0.3)

# S-wave time differences (relative to IASP91)
ax = axes[1, 1]
iasp91_s = travel_time_data['iasp91']['s_times']
for (model, data), color in zip(travel_time_data.items(), colors):
    if model != 'iasp91':
        diff = data['s_times'] - iasp91_s
        ax.plot(distances, diff, color=color, linewidth=2,
               marker='o', markersize=4, label=f'{model.upper()} - IASP91')

ax.axhline(0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Distance (degrees)')
ax.set_ylabel('S-wave Time Difference (s)')
ax.set_title('S-wave Time Differences (relative to IASP91)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Statistical Analysis of Model Differences

Let's quantify the differences between models statistically.

In [ ]:
# Calculate statistics for model differences
print("Travel Time Differences (relative to IASP91)")
print("=" * 60)

for model in models:
    if model != 'iasp91':
        print(f"\n{model.upper()} vs IASP91:")

        # P-wave differences
        p_diff = travel_time_data[model]['p_times'] - travel_time_data['iasp91']['p_times']
        print(f"  P-wave differences:")
        print(f"    Mean: {np.mean(p_diff):.2f} s")
        print(f"    Std:  {np.std(p_diff):.2f} s")
        print(f"    Range: {np.min(p_diff):.2f} to {np.max(p_diff):.2f} s")

        # S-wave differences
        s_diff = travel_time_data[model]['s_times'] - travel_time_data['iasp91']['s_times']
        print(f"  S-wave differences:")
        print(f"    Mean: {np.mean(s_diff):.2f} s")
        print(f"    Std:  {np.std(s_diff):.2f} s")
        print(f"    Range: {np.min(s_diff):.2f} to {np.max(s_diff):.2f} s")

        # Maximum absolute differences
        print(f"  Maximum absolute differences:")
        print(f"    P-wave: {np.max(np.abs(p_diff)):.2f} s")
        print(f"    S-wave: {np.max(np.abs(s_diff)):.2f} s")

## 5. Ray Path Comparison Between Models

Let's see how ray paths differ between models at a specific distance.

In [ ]:
# Compare ray paths between models
distance = 60  # degrees
source_depth = 10  # km

fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Plot Earth circle
from matplotlib.patches import Circle
earth_circle = Circle((0, 0), 6371, fill=False, color='black', linewidth=2)
ax.add_patch(earth_circle)

cmb_circle = Circle((0, 0), 3480, fill=False, color='gray', linewidth=1, linestyle='--')
ax.add_patch(cmb_circle)

colors = ['blue', 'red', 'green']
ray_stats = {}

for (model, color) in zip(models, colors):
    tracer = RayPathTracer(model)
    rays = tracer.get_ray_paths(source_depth, distance, phases=['P'])

    if rays:
        ray = rays[0]
        distances_rad = np.radians(ray.path['dist'])
        radii = 6371 - ray.path['depth']

        x = radii * np.sin(distances_rad)
        y = radii * np.cos(distances_rad)

        ax.plot(x, y, color=color, linewidth=2,
               label=f'{model.upper()} ({ray.time:.1f} s)')

        # Store ray statistics
        ray_stats[model] = {
            'travel_time': ray.time,
            'max_depth': np.max(ray.path['depth']),
            'ray_param': ray.ray_param
        }

# Plot source and receiver
source_x = (6371 - source_depth) * np.sin(0)
source_y = (6371 - source_depth) * np.cos(0)
ax.plot(source_x, source_y, 'k*', markersize=15, label='Source')

receiver_x = 6371 * np.sin(np.radians(distance))
receiver_y = 6371 * np.cos(np.radians(distance))
ax.plot(receiver_x, receiver_y, 'k^', markersize=12, label='Receiver')

ax.set_xlim(-7000, 7000)
ax.set_ylim(-7000, 7000)
ax.set_aspect('equal')
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Distance (km)')
ax.set_title(f'P-wave Ray Paths: Model Comparison (Distance = {distance}°, Depth = {source_depth} km)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

# Print ray statistics
print(f"\nRay Path Statistics for {distance}° distance:")
print("-" * 50)
print(f"{'Model':<8} {'Time (s)':<10} {'Max Depth (km)':<15} {'Ray Param':<12}")
print("-" * 50)
for model, stats in ray_stats.items():
    print(f"{model.upper():<8} {stats['travel_time']:<10.1f} {stats['max_depth']:<15.1f} {stats['ray_param']:<12.3f}")

## 6. Model Performance for Different Source Depths

Let's examine how model differences vary with source depth.

In [ ]:
# Compare models at different source depths
depths = [0, 10, 50, 100, 200, 400]  # km
distance = 60  # degrees

depth_comparison = {}

for model in models:
    calc = TravelTimeCalculator(model)
    p_times = []
    s_times = []

    for depth in depths:
        arrivals = calc.calculate_travel_times(depth, distance)

        # Find P and S arrivals
        p_time = None
        s_time = None

        for arrival in arrivals:
            if arrival.name == 'P' and p_time is None:
                p_time = arrival.time
            elif arrival.name == 'S' and s_time is None:
                s_time = arrival.time

        p_times.append(p_time)
        s_times.append(s_time)

    depth_comparison[model] = {'p_times': np.array(p_times), 's_times': np.array(s_times)}

# Plot depth comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
colors = ['blue', 'red', 'green']

# P-wave travel times vs depth
ax = axes[0]
for (model, data), color in zip(depth_comparison.items(), colors):
    ax.plot(depths, data['p_times']/60, color=color, linewidth=2,
           marker='o', markersize=6, label=model.upper())

ax.set_xlabel('Source Depth (km)')
ax.set_ylabel('P-wave Travel Time (minutes)')
ax.set_title(f'P-wave Travel Times vs Source Depth (Distance = {distance}°)')
ax.legend()
ax.grid(True, alpha=0.3)

# S-wave travel times vs depth
ax = axes[1]
for (model, data), color in zip(depth_comparison.items(), colors):
    ax.plot(depths, data['s_times']/60, color=color, linewidth=2,
           marker='o', markersize=6, label=model.upper())

ax.set_xlabel('Source Depth (km)')
ax.set_ylabel('S-wave Travel Time (minutes)')
ax.set_title(f'S-wave Travel Times vs Source Depth (Distance = {distance}°)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate depth-dependent differences
print(f"\nTravel Time Differences vs Source Depth (Distance = {distance}°):")
print("=" * 70)
print(f"{'Depth (km)':<12} {'PREM-IASP91 (s)':<15} {'AK135-IASP91 (s)':<15}")
print(f"{'':12} {'P':<7} {'S':<7} {'P':<7} {'S':<7}")
print("-" * 70)

for i, depth in enumerate(depths):
    prem_p_diff = depth_comparison['prem']['p_times'][i] - depth_comparison['iasp91']['p_times'][i]
    prem_s_diff = depth_comparison['prem']['s_times'][i] - depth_comparison['iasp91']['s_times'][i]
    ak135_p_diff = depth_comparison['ak135']['p_times'][i] - depth_comparison['iasp91']['p_times'][i]
    ak135_s_diff = depth_comparison['ak135']['s_times'][i] - depth_comparison['iasp91']['s_times'][i]

    print(f"{depth:<12} {prem_p_diff:<7.2f} {prem_s_diff:<7.2f} {ak135_p_diff:<7.2f} {ak135_s_diff:<7.2f}")

## Summary

In this notebook, we demonstrated:

1. **Earth model exploration** using the `EarthModelManager` class
2. **Velocity structure comparison** between iasp91, prem, and ak135 models
3. **Travel time differences** for P and S waves across different models
4. **Statistical analysis** of model differences
5. **Ray path comparison** showing geometric differences between models
6. **Depth-dependent analysis** of model performance

### Key Findings:
- Different Earth models show subtle but systematic differences in velocity structure
- Travel time differences between models are typically within a few seconds
- PREM and AK135 generally show small differences compared to IASP91
- Model differences can vary with both distance and source depth
- Ray path geometries are very similar between models, with slight differences in timing
- The choice of Earth model can impact seismic analysis, especially for high-precision applications

### Practical Implications:
- For regional studies, model choice may be important
- IASP91 remains a good general-purpose model
- PREM provides more detailed structure and is widely used
- AK135 offers improvements for specific applications
- Model uncertainty should be considered in seismic analysis

The `seisray` package makes it easy to compare different Earth models and assess their impact on seismic wave propagation calculations.